In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%pip install -q -r requirements.txt

Obtaining file:///home/jovyan/workspace/untitled1-vector-search/backend (from -r requirements.txt (line 1))
  Attempting uninstall: arXiv-vecsim
    Found existing installation: arXiv-vecsim 0.0.1
    Uninstalling arXiv-vecsim-0.0.1:
      Successfully uninstalled arXiv-vecsim-0.0.1
  Running setup.py develop for arXiv-vecsim
Note: you may need to restart the kernel to use updated packages.


In [3]:
import json
import pandas as pd
import os
import re
import string
from transformers import BertForSequenceClassification
from sklearn.preprocessing import MultiLabelBinarizer
from transformers import AutoTokenizer
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
from transformers import TrainingArguments, Trainer
from datasets import Dataset
import numpy as np
from sklearn.model_selection import train_test_split

from vecsim_app.categories import CATEGORIES
from vecsim_app.data_utils import papers

DATA_PATH = "/home/jovyan/arxiv/arxiv-metadata-oai-snapshot.json"
YEAR_CUTOFF = 2012
YEAR_PATTERN = r"(19|20[0-9]{2})"

In [4]:
df = pd.DataFrame(papers(data_path=DATA_PATH, year_cutoff=YEAR_CUTOFF, year_pattern=YEAR_PATTERN))
len(df)

408773

In [5]:
df.head(3)

,id,title,year,authors,categories,abstract
0,0704.0304,The World as Evolving Information,2012,Carlos Gershenson,"cs.IT,cs.AI,math.IT,q-bio.PE",This paper discusses the benefits of describ...
1,0704.2744,Nahm transform and parabolic minimal Laplace t...,2012,Szilard Szabo,math.AG,We prove that Nahm transform for integrable ...
2,0704.2768,Heat Equations and the Weighted $\bar\partial$...,2012,Andrew Raich,"math.AP,math.CV",The purpose of this article is to establish ...


In [6]:
df['text'] = df['title'] + ' ' + df['abstract']
# df['categories'] = df['categories'].apply(lambda x: x.split(','))

In [7]:
df.iloc[0].categories

'cs.IT,cs.AI,math.IT,q-bio.PE'

In [8]:
# Split into train and test
df_train, df_test = train_test_split(df, test_size=0.2)

In [9]:
def get_tokenizer(tokenizer_model):
    def tokenize_function(examples):
        return tokenizer(examples["text"], padding="max_length", truncation=True)

    tokenizer = AutoTokenizer.from_pretrained(tokenizer_model)
    return tokenize_function, tokenizer

tokenize_function, tokenizer = get_tokenizer('bert-base-uncased')

In [10]:
mlb = MultiLabelBinarizer()
mlb.fit(df_train['categories'])

MultiLabelBinarizer()

In [11]:
def preprocess_data(examples):
  # take a batch of texts
  text = examples["text"]

  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)

  encoded_categories = mlb.transform(examples['categories']).astype(float)

  encoding["labels"] = encoded_categories

  return encoding

In [12]:
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", 
    num_labels=len(mlb.classes_), 
    problem_type="multi_label_classification"
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [13]:
df_train_hf = Dataset.from_pandas(df_train[['text', 'categories']].head(100))
tokenized_train = df_train_hf.map(preprocess_data, batched=True)

df_test_hf = Dataset.from_pandas(df_test[['text', 'categories']].head(100))
tokenized_test = df_test_hf.map(preprocess_data, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [14]:
# Debugging - get inverse transform

print("Reversed", mlb.inverse_transform(np.asarray(tokenized_test[0]['labels']).reshape(1, -1)))
print("Original categories", tokenized_test[0]['categories'])

Reversed [(',', '.', 'A', 'D', 'G', 'N', 'P', 'a', 'h', 'm', 't')]
Original categories math.NA,math.AP,math.DG


In [29]:
# Adaptation: https://github.com/NielsRogge/Transformers-Tutorials/blob/master/BERT/Fine_tuning_BERT_(and_friends)_for_multi_label_text_classification.ipynb 
# 
batch_size = 16
metric_name = "f1"

args = TrainingArguments(
    f"paper-multilabel-finetuning",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)


# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [30]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [31]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, categories, __index_level_0__. If text, categories, __index_level_0__ are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/srv/conda/envs/saturn/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 100
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 35


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,No log,0.454203,0.611085,0.750968,0.000000
2,No log,0.435303,0.596564,0.732773,0.000000
3,No log,0.429763,0.592554,0.729754,0.000000
4,No log,0.429101,0.620622,0.752786,0.000000
5,No log,0.427151,0.607813,0.741955,0.000000


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, categories, __index_level_0__. If text, categories, __index_level_0__ are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 100
  Batch size = 16
Saving model checkpoint to paper-multilabel-finetuning/checkpoint-7
Configuration saved in paper-multilabel-finetuning/checkpoint-7/config.json
Model weights saved in paper-multilabel-finetuning/checkpoint-7/pytorch_model.bin
tokenizer config file saved in paper-multilabel-finetuning/checkpoint-7/tokenizer_config.json
Special tokens file saved in paper-multilabel-finetuning/checkpoint-7/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, categories, __index_level_0__. If text, cate

TrainOutput(global_step=35, training_loss=0.43726703098842074, metrics={'train_runtime': 45.6102, 'train_samples_per_second': 10.962, 'train_steps_per_second': 0.767, 'total_flos': 32901874944000.0, 'train_loss': 0.43726703098842074, 'epoch': 5.0})

In [34]:
eval = trainer.evaluate()
eval

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, categories, __index_level_0__. If text, categories, __index_level_0__ are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 100
  Batch size = 16


{'eval_loss': 0.42910125851631165,
 'eval_f1': 0.6206216082881105,
 'eval_roc_auc': 0.7527857944524611,
 'eval_accuracy': 0.0,
 'eval_runtime': 0.7334,
 'eval_samples_per_second': 136.352,
 'eval_steps_per_second': 9.545,
 'epoch': 5.0}

In [35]:
text = df['text'].iloc[5]

encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k, v in encoding.items()}

outputs = trainer.model(**encoding)
logits = outputs.logits

In [36]:
# apply sigmoid + threshold
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.1)] = 1

In [37]:
print(text)
print(mlb.inverse_transform(predictions.reshape(1, -1)))

Suppression of growth by multiplicative white noise in a parametric
  resonant system   The author studied the growth of the amplitude in a Mathieu-like equation
with multiplicative white noise. The approximate value of the exponent at the
extremum on parametric resonance regions was obtained theoretically by
introducing the width of time interval, and the exponents were calculated
numerically by solving the stochastic differential equations by a symplectic
numerical method. The Mathieu-like equation contains a parameter $\alpha$ that
is determined by the intensity of noise and the strength of the coupling
between the variable and the noise. The value of $\alpha$ was restricted not to
be negative without loss of generality. It was shown that the exponent
decreases with $\alpha$, reaches a minimum and increases after that. It was
also found that the exponent as a function of $\alpha$ has only one minimum at
$\alpha \neq 0$ on parametric resonance regions of $\alpha = 0$. This minimum
va

In [41]:
import pickle


with open('mlb.pkl', 'wb') as handle:
    pickle.dump(mlb, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [39]:
with open('mlb_metrics.json', 'w') as f:
    f.write(json.dumps(eval, indent=4))

In [40]:
trainer.save_model(output_dir='./checkpoint')

Saving model checkpoint to ./checkpoint
Configuration saved in ./checkpoint/config.json
Model weights saved in ./checkpoint/pytorch_model.bin
tokenizer config file saved in ./checkpoint/tokenizer_config.json
Special tokens file saved in ./checkpoint/special_tokens_map.json
